In [ ]:
import cv2 as cv
import numpy as np
from skimage.feature import hog, local_binary_pattern


def hog_extraction(img):
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    hog_features = hog(
        gray_img,
        orientations=8,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
    )
    return hog_features.flatten()

def color_hist_extraction(img, bins=32):
    hist_features = []
    for ch, ranges in zip(
        [0, 1, 2],
        [(0,180), (0,256), (0,256)]
    ):
        hist = cv.calcHist([img], [ch], None, [bins], ranges)
        hist = cv.normalize(hist, hist).flatten()
        hist_features.extend(hist)
    return np.array(hist_features)

def lbp_extraction(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    radius = 1
    n_points = 8
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
    n_bins = n_points + 2

    lbp_features, _ = np.histogram(
        lbp.ravel(),
        bins=n_bins,
        range=(0, n_bins),
        density=True
    )
    return lbp_features.flatten()

In [ ]:
import numpy as np

def feature_extraction(img):
    hog_feature = hog_extraction(img)
    color_hist_feature = color_hist_extraction(img)
    lbp_feature = lbp_extraction(img)

    feature_vector = np.concatenate(
        [hog_feature, color_hist_feature, lbp_feature]
    )
    
    return feature_vector

In [ ]:
# Extract all features from the dataset

import os
import cv2 as cv
import matplotlib.pyplot as plt

DATASET_SRC = "../aug_dataset"

x = []
y = []

y_labals = {
    "glass": 0,
    "paper": 1,
    "cardboard": 2,
    "plastic": 3,
    "metal": 4,
    "trash": 5
}

for class_name in os.listdir(DATASET_SRC):
    src = os.path.join(DATASET_SRC, class_name)
    images = os.listdir(src)

    for image_name in images:
        img_path = os.path.join(src, image_name)
        img = cv.imread(img_path)

        if img is None:
            continue
        
        feature = feature_extraction(img)
        x.append(feature)
        y.append(y_labals[class_name])
        


x = np.array(x)
y = np.array(y)

print(f"x's shape: {x.shape}")
print(f"y's shape: {y.shape}")

# Save to disk
np.savez("../features_dataset.npz", X=x, y=y)
        


In [ ]:
# Load the feature dataset

import numpy as np

data = np.load("../features_dataset.npz")

X = data["X"]
y = data["y"]

print(f"x's shape: {X.shape}")
print(f"y's shape: {y.shape}")
